# Validation and cross-validation

In this exercise you will implement a validation pipeline.

At the end of the MSLE exercise you tested your model against the training and test datasets. As you should observe, there's a gap between the results. By validating your model, not only should you be able to anticipate the test time performance, but also have a method to compare different models.

Implement the basic validation method, i.e. a random split. Test it with your model from Exercise MSLE.

In [1]:
%matplotlib inline

#!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
#!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

In [2]:
from typing import Tuple

import numpy as np
import pandas as pd
from sklearn import preprocessing

np.random.seed(357)

In [3]:
def load(name: str) -> Tuple[np.ndarray, np.array]:
    data = pd.read_csv(name)
    x = data.loc[:, data.columns != 'cena'].to_numpy()
    y = data['cena'].to_numpy()

    return x, y

In [4]:
x_train, y_train = load('mieszkania.csv')
x_test, y_test = load('mieszkania_test.csv')

x_test, y_test

(array([[71, 'wolowo', 2, 2, 1912, 1],
        [45, 'mokotowo', 1, 1, 1938, 0],
        [38, 'mokotowo', 1, 1, 1999, 1],
        ...,
        [89, 'wolowo', 2, 2, 1922, 1],
        [40, 'wolowo', 1, 1, 1959, 0],
        [68, 'grodziskowo', 2, 1, 1927, 0]], dtype=object),
 array([ 322227,  295878,  306530,  553641,  985348,  695726,   99751,
         891261,  536499,  527093,  861472,  701472,  429776,  547725,
         669560,  318362, 1140170,  341242,  113580,  456093,  470730,
         421012,  617318,  796117,  138901,  857820,  939450,  398165,
         944399, 1025413,  522440,  344346,  145702,  246712,  574154,
         807608,  568048,  412494,  588840,  766040,  979540, 1044803,
         742235,  758936,  388672,  178238,  530053, 1150687,  587013,
         269316,  270969, 1008103,  299708,  393925,  511106,  947932,
         127717,  752428, 1185932,  330988,  330699,  403778,  584561,
         795392,  602356,  680512,  202121,  888872,  456054,  227841,
         343730,  

In [13]:
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(x_train[:, 1])
x_train[:, 1] = labelencoder.transform(x_train[:, 1])
x_test[:, 1] = labelencoder.transform(x_test[:, 1])

x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)
x_train

array([[1.040e+02, 1.000e+00, 2.000e+00, 2.000e+00, 1.940e+03, 1.000e+00],
       [4.300e+01, 2.000e+00, 1.000e+00, 1.000e+00, 1.970e+03, 1.000e+00],
       [1.280e+02, 0.000e+00, 3.000e+00, 2.000e+00, 1.916e+03, 1.000e+00],
       ...,
       [1.070e+02, 0.000e+00, 2.000e+00, 2.000e+00, 1.935e+03, 0.000e+00],
       [1.170e+02, 0.000e+00, 3.000e+00, 2.000e+00, 1.978e+03, 1.000e+00],
       [5.600e+01, 3.000e+00, 2.000e+00, 1.000e+00, 1.923e+03, 0.000e+00]])

In [14]:
import plotly.express as px

def plot_loss(l):
    fig = px.line(y=l, labels={'y':'loss'})
    fig.show()

def mse(xs, ps):
    assert len(xs) == len(ps)

    n = len(xs)
    sum = np.sum((xs - ps) ** 2)

    return (1/n) * sum

def msle(xs, ps):
    assert len(xs) == len(ps)

    n = len(xs)
    sum = np.sum((np.log(1 + xs) - np.log(1 + ps)) ** 2)

    return (1/n) * sum

In [15]:
n, features = x_train.shape
lr = 1e-2 # step size
n_epochs = 40


In [ ]:
def predict(w, xs):
    return xs @ w

def evaluate(w, xs, ys):
    # print(w, "\n", xs, "\n", ys)
    # print("\n\n\n\n\n")
    return msle(ys, predict(w, xs))

def linear_regression(n_epochs, lr, x, y):
    n, features = x.shape
    w = np.zeros(features).reshape(-1, 1)
    losses = [evaluate(w, x, y)]

    for i in range(n_epochs):
        y_hat = predict(w, x)
        dJdwi = np.sum((y_hat - y) * x, axis = 0)

        w = w - (2/n) * (lr * dJdwi).reshape(-1,1)

        loss = evaluate(w, x, y)
        losses.append(loss)

        if i == 0 or (i+1) % 5 == 0:
            print(f'Iter: {i:>3} Loss: {loss:8.8f} w: {[f"{x[0]:.2f}" for x in w]}')

    return (w, losses)

def linear_regression_log(n_epochs, lr, x, y):
    return linear_regression(n_epochs, lr, np.log(1 + x), np.log(1 + y))

(w_msle, losses) = linear_regression_log(n_epochs, lr, x_train, y_train)

# print(w_msle)
# plot_loss(losses)
# evaluate(w_msle, x_test, y_test)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] 
 [[4.65396035 0.69314718 1.09861229 1.09861229 7.57095858 0.69314718]
 [3.78418963 1.09861229 0.69314718 0.69314718 7.58629631 0.69314718]
 [4.8598124  0.         1.38629436 1.09861229 7.55851674 0.69314718]
 ...
 [4.68213123 0.         1.09861229 1.09861229 7.56837927 0.        ]
 [4.77068462 0.         1.38629436 1.09861229 7.59034695 0.69314718]
 [4.04305127 1.38629436 1.09861229 0.69314718 7.56216163 0.        ]] 
 [[13.56717099]
 [12.75682931]
 [13.16822927]
 [13.63034416]
 [13.90212852]
 [13.30311871]
 [13.04686367]
 [12.02446683]
 [12.67247605]
 [13.82684211]
 [12.97029087]
 [13.86583193]
 [13.24187647]
 [13.3382003 ]
 [13.90879869]
 [13.37731718]
 [13.56206629]
 [12.81546587]
 [12.45757428]
 [13.82546386]
 [12.85211512]
 [13.79588014]
 [12.58364912]
 [13.57752871]
 [13.8782149 ]
 [13.53843998]
 [13.38784432]
 [13.15204256]
 [13.13236321]
 [13.08125384]
 [13.21931584]
 [11.63191666]
 [12.58165226]
 [11.7621967 ]
 [13.35212975]
 [13.08748618]

In [9]:
#######################################################
# TODO: Implement the basic validation method,        #
# compare MSLE on training, validation, and test sets #
#######################################################
v_epochs = np.linspace(30, 80, 10).astype(int).reshape(-1,1)
v_lr = np.linspace(1e-2, 1e-4, 50).reshape(-1, 1)

print(v_epochs.shape, v_lr.shape)

xepochs, ylr = np.meshgrid(v_epochs, v_lr, indexing='ij')

def linreg_w(n_epochs, lr, x, y):
    return linear_regression_log(n_epochs, lr, x, y)[0]

def shuffle_and_divide(length, ratio):
    perm = np.random.permutation(length)
    return (perm[: int(length * ratio)], perm[int(length * ratio) :])

idx_train, idx_val = shuffle_and_divide(n, 0.8)

x_train_, x_val = x_train[idx_train], x_train[idx_val]
y_train_, y_val = y_train[idx_train], y_train[idx_val]

run_linreg = np.frompyfunc(lambda ne, lr: linreg_w(ne, lr, x_train_, y_train_), 2, 1)

import contextlib, io
with contextlib.redirect_stdout(io.StringIO()):
    w_matrix = run_linreg(xepochs, ylr)

check_loss = np.frompyfunc(lambda w : evaluate(w, x_val, y_val), 1, 1)

loss_matrix = check_loss(w_matrix)

print(np.min(loss_matrix))
idx = np.unravel_index(np.argmin(loss_matrix), xepochs.shape)

found_w = w_matrix[idx]
print(xepochs[idx], ylr[idx], w_matrix[idx])

evaluate(found_w, x_val, y_val)
#def split_validation():
    

(10, 1) (50, 1)
26.609151953251935
30 0.001716326530612244 [[0.71008659]
 [0.13825149]
 [0.17719597]
 [0.15688876]
 [1.25179597]
 [0.0554241 ]]


np.float64(26.609151953251935)

To make the random split validation reliable, a huge chunk of training data may be needed. To get over this problem, one may apply cross-validaiton.

![alt-text](https://chrisjmccormick.files.wordpress.com/2013/07/10_fold_cv.png)

Let's now implement the method. Make sure that:
* number of partitions is a parameter,
* the method is not limited to `mieszkania.csv`,
* the method is not limited to one specific model.

In [10]:
####################################
# TODO: Implement cross-validation #
####################################

Recall that sometimes validation may be tricky, e.g. significant class imbalance, having a small number of subjects, geographically clustered instances...

What could in theory go wrong here with random, unstratified partitions? Think about potential solutions and investigate the data in order to check whether these problems arise here.

In [11]:
##############################
# TODO: Investigate the data #
##############################